In [30]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [31]:
IMAGE_SIZE=224
train_ds = tf.keras.utils.image_dataset_from_directory(directory='..\original_frames',
      image_size=(IMAGE_SIZE, IMAGE_SIZE),
      seed=37,validation_split=0.2, subset="training") 
val_ds = tf.keras.utils.image_dataset_from_directory(directory='..\original_frames',
      image_size=(IMAGE_SIZE, IMAGE_SIZE),
      seed=37,validation_split=0.2, subset="validation") 
class_names=train_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 252 files belonging to 7 classes.
Using 202 files for training.
Found 252 files belonging to 7 classes.
Using 50 files for validation.
['BEAUTIFUL', 'HOT', 'LIKE', 'SUBJECT_I', 'SUMMER', 'SWIM', 'WINTER']


In [38]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

In [39]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [40]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 batch_normalization_5 (Batc  (None, 54, 54, 96)       384       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 26, 26, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 batch_normalization_6 (Batc  (None, 26, 26, 256)      1024      
 hNormalization)                                                 
                                                      

In [41]:
model.fit(train_ds,
          epochs=50,
          validation_data=val_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

Epoch 1/50
7/7 [==============================] - 5s 609ms/step - loss: 2.9217 - accuracy: 0.2673 - val_loss: 7.9639 - val_accuracy: 0.1400
Epoch 2/50
7/7 [==============================] - 4s 584ms/step - loss: 1.2857 - accuracy: 0.6238 - val_loss: 2.4247 - val_accuracy: 0.1400
Epoch 3/50
7/7 [==============================] - 4s 619ms/step - loss: 0.6258 - accuracy: 0.7772 - val_loss: 2.0753 - val_accuracy: 0.2800
Epoch 4/50
7/7 [==============================] - 4s 622ms/step - loss: 0.3126 - accuracy: 0.9010 - val_loss: 1.9004 - val_accuracy: 0.3200
Epoch 5/50
7/7 [==============================] - 5s 637ms/step - loss: 0.1725 - accuracy: 0.9356 - val_loss: 1.7148 - val_accuracy: 0.3200
Epoch 6/50
7/7 [==============================] - 5s 681ms/step - loss: 0.1217 - accuracy: 0.9703 - val_loss: 1.5946 - val_accuracy: 0.3400
Epoch 7/50
7/7 [==============================] - 5s 665ms/step - loss: 0.1185 - accuracy: 0.9653 - val_loss: 1.4092 - val_accuracy: 0.4000
Epoch 8/50
7/7 [====

In [44]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs